In [1]:
from pathlib import Path
import torch

cmip_path = Path("/data0/datasets/weatherbench/data/weatherbench/cmip6/5.625")   # replace with path
era_path = Path("/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg")

In [2]:
import xarray as xr

import sys
sys.path.insert(0, '/home/seongbin/climate-learn/src')

In [3]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import DataModule

In [4]:
era5_data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = era_path,
    in_vars = ["temperature"],
    out_vars = ["temperature_850"],
    train_start_year = Year(2007),
    val_start_year = Year(2011),
    test_start_year = Year(2013),
    end_year = Year(2014),
    pred_range = Days(5),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 16
) 

Creating train dataset
update


100%|█| 4/4 [00:00<00:00, 96.6

out 35064


in 35064
Finished inp and out _data
True
Almost done!
Creating val dataset
update


100%|█| 2/2 [00:00<00:00, 135.


out 17544
in 17544
Finished inp and out _data
True
Almost done!
Creating test dataset
update


100%|█| 2/2 [00:00<00:00, 158.

out 17520


in 17520
Finished inp and out _data
True
Almost done!


In [5]:
cmip6_data_module = DataModule(
    dataset = "CMIP6",
    task = "forecasting",
    root_dir = cmip_path,
    in_vars = ["temperature"],
    out_vars = ["temperature_850"],
    train_start_year = Year(2000),
    val_start_year = Year(2011),
    test_start_year = Year(2013),
    end_year = Year(2014),
    pred_range = Days(5),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 16
) 

Creating train dataset


  0%|          | 0/11 [00:00<?, ?it/s]

in 16069
out 16069
Creating val dataset


  0%|          | 0/2 [00:00<?, ?it/s]

in 2918
out 2918
Creating test dataset


  0%|          | 0/2 [00:00<?, ?it/s]

in 2918
out 2918


In [6]:
from climate_learn.models import load_model

In [7]:
cmip_model_kwargs = {
    "in_channels": len(cmip6_data_module.hparams.in_vars),
    "out_channels": len(cmip6_data_module.hparams.out_vars),
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 2,
}

cmip_model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = cmip_model_kwargs, optim_kwargs = optim_kwargs)

In [15]:
from climate_learn.models import set_climatology
from climate_learn.training import Trainer, WandbLogger

In [ ]:
set_climatology(cmip_model_module, cmip6_data_module)

In [10]:
cmip_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [11]:
cmip_trainer.fit(cmip_model_module, cmip6_data_module)

/home/seongbin/miniconda3/envs/climate_pretrain/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /data0/ckpts/seongbin/cross-train-z500/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                       ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                        │ ResNet            │  1.2 M │
│ 1  │ net.activation             │ LeakyReLU         │      0 │
│ 2  │ net.image_proj             │ PeriodicConv2D    │  6.4 K │
│ 3  │ net.image_proj.padding     │ PeriodicPadding2D │      0 │
│ 4  │ net.image_proj.conv        │ Conv2d            │  6.4 K │
│ 5  │ net.blocks                 │ ModuleList        │  1.2 M │
│ 6  │ net.blocks.0               │ ResidualBlock     │  295 K │
│ 7  │ net.blocks.0.activation    │ LeakyReLU         │      0 │
│ 8  │ net.blocks.0.conv1         │ PeriodicConv2D    │  147 K │
│ 9  │ net.blocks.0.conv1.padding │ PeriodicPadding2D │      0 │
│ 10 │ net.blocks.0.conv1.conv    │ Conv2d            │  147 K │
│ 11 │ net.blocks.0.conv2         │ PeriodicConv2D    │  147 K │
│ 12 │ net.blocks.0.conv2.padding │ PeriodicPadding2D │      0 │
│ 13 │ net.blocks.0.conv2.conv    │ Conv2d            │  147 K │
│ 14 │ net.blocks.0.shortcut      │ Identity          │      0 │
│ 15 │ net.blocks.0.norm1         │ BatchNorm2d       │    256 │
│ 16 │ net.blocks.0.norm2         │ BatchNorm2d       │    256 │
│ 17 │ net.blocks.0.drop          │ Dropout           │      0 │
│ 18 │ net.blocks.1               │ ResidualBlock     │  295 K │
│ 19 │ net.blocks.1.activation    │ LeakyReLU         │      0 │
│ 20 │ net.blocks.1.conv1         │ PeriodicConv2D    │  147 K │
│ 21 │ net.blocks.1.conv1.padding │ PeriodicPadding2D │      0 │
│ 22 │ net.blocks.1.conv1.conv    │ Conv2d            │  147 K │
│ 23 │ net.blocks.1.conv2         │ PeriodicConv2D    │  147 K │
│ 24 │ net.blocks.1.conv2.padding │ PeriodicPadding2D │      0 │
│ 25 │ net.blocks.1.conv2.conv    │ Conv2d            │  147 K │
│ 26 │ net.blocks.1.shortcut      │ Identity          │      0 │
│ 27 │ net.blocks.1.norm1         │ BatchNorm2d       │    256 │
│ 28 │ net.blocks.1.norm2         │ BatchNorm2d       │    256 │
│ 29 │ net.blocks.1.drop          │ Dropout           │      0 │
│ 30 │ net.blocks.2               │ ResidualBlock     │  295 K │
│ 31 │ net.blocks.2.activation    │ LeakyReLU         │      0 │
│ 32 │ net.blocks.2.conv1         │ PeriodicConv2D    │  147 K │
│ 33 │ net.blocks.2.conv1.padding │ PeriodicPadding2D │      0 │
│ 34 │ net.blocks.2.conv1.conv    │ Conv2d            │  147 K │
│ 35 │ net.blocks.2.conv2         │ PeriodicConv2D    │  147 K │
│ 36 │ net.blocks.2.conv2.padding │ PeriodicPadding2D │      0 │
│ 37 │ net.blocks.2.conv2.conv    │ Conv2d            │  147 K │
│ 38 │ net.blocks.2.shortcut      │ Identity          │      0 │
│ 39 │ net.blocks.2.norm1         │ BatchNorm2d       │    256 │
│ 40 │ net.blocks.2.norm2         │ BatchNorm2d       │    256 │
│ 41 │ net.blocks.2.drop          │ Dropout           │      0 │
│ 42 │ net.blocks.3               │ ResidualBlock     │  295 K │
│ 43 │ net.blocks.3.activation    │ LeakyReLU         │      0 │
│ 44 │ net.blocks.3.conv1         │ PeriodicConv2D    │  147 K │
│ 45 │ net.blocks.3.conv1.padding │ PeriodicPadding2D │      0 │
│ 46 │ net.blocks.3.conv1.conv    │ Conv2d            │  147 K │
│ 47 │ net.blocks.3.conv2         │ PeriodicConv2D    │  147 K │
│ 48 │ net.blocks.3.conv2.padding │ PeriodicPadding2D │      0 │
│ 49 │ net.blocks.3.conv2.conv    │ Conv2d            │  147 K │
│ 50 │ net.blocks.3.shortcut      │ Identity          │      0 │
│ 51 │ net.blocks.3.norm1         │ BatchNorm2d       │    256 │
│ 52 │ net.blocks.3.norm2         │ BatchNorm2d       │    256 │
│ 53 │ net.blocks.3.drop          │ Dropout           │      0 │
│ 54 │ net.norm                   │ BatchNorm2d       │    256 │
│ 55 │ net.final                  │ PeriodicConv2D    │  6.3 K │
│ 56 │ net.final.padding          │ PeriodicPadding2D │      0 │
│ 57 │ net.final.conv             │ Conv2d            │  6.3 K │
│ 58 │ denormalization            │

Trainable params: 1.2 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.2 M                                                                          
Total estimated model params size (MB): 2

Output()

In [12]:
from climate_learn.models import fit_lin_reg_baseline
fit_lin_reg_baseline(cmip_model_module, cmip6_data_module, reg_hparam=0.0)

## Train on ERA5

In [9]:
era5_model_kwargs = {
    "in_channels": len(era5_data_module.hparams.in_vars),
    "out_channels": len(era5_data_module.hparams.out_vars),
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

era5_model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = era5_model_kwargs, optim_kwargs = optim_kwargs)

In [17]:
set_climatology(era5_model_module, era5_data_module)

In [18]:
era5_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [19]:
era5_trainer.fit(era5_model_module, era5_data_module)

/home/seongbin/miniconda3/envs/climate_pretrain/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /data0/ckpts/seongbin/cross-train-z500/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                       ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                        │ ResNet            │  1.2 M │
│ 1  │ net.activation             │ LeakyReLU         │      0 │
│ 2  │ net.image_proj             │ PeriodicConv2D    │  6.4 K │
│ 3  │ net.image_proj.padding     │ PeriodicPadding2D │      0 │
│ 4  │ net.image_proj.conv        │ Conv2d            │  6.4 K │
│ 5  │ net.blocks                 │ ModuleList        │  1.2 M │
│ 6  │ net.blocks.0               │ ResidualBlock     │  295 K │
│ 7  │ net.blocks.0.activation    │ LeakyReLU         │      0 │
│ 8  │ net.blocks.0.conv1         │ PeriodicConv2D    │  147 K │
│ 9  │ net.blocks.0.conv1.padding │ PeriodicPadding2D │      0 │
│ 10 │ net.blocks.0.conv1.conv    │ Conv2d            │  147 K │
│ 11 │ net.blocks.0.conv2         │ PeriodicConv2D    │  147 K │
│ 12 │ net.blocks.0.conv2.padding │ PeriodicPadding2D │      0 │
│ 13 │ net.blocks.0.conv2.conv    │ Conv2d            │  147 K │
│ 14 │ net.blocks.0.shortcut      │ Identity          │      0 │
│ 15 │ net.blocks.0.norm1         │ BatchNorm2d       │    256 │
│ 16 │ net.blocks.0.norm2         │ BatchNorm2d       │    256 │
│ 17 │ net.blocks.0.drop          │ Dropout           │      0 │
│ 18 │ net.blocks.1               │ ResidualBlock     │  295 K │
│ 19 │ net.blocks.1.activation    │ LeakyReLU         │      0 │
│ 20 │ net.blocks.1.conv1         │ PeriodicConv2D    │  147 K │
│ 21 │ net.blocks.1.conv1.padding │ PeriodicPadding2D │      0 │
│ 22 │ net.blocks.1.conv1.conv    │ Conv2d            │  147 K │
│ 23 │ net.blocks.1.conv2         │ PeriodicConv2D    │  147 K │
│ 24 │ net.blocks.1.conv2.padding │ PeriodicPadding2D │      0 │
│ 25 │ net.blocks.1.conv2.conv    │ Conv2d            │  147 K │
│ 26 │ net.blocks.1.shortcut      │ Identity          │      0 │
│ 27 │ net.blocks.1.norm1         │ BatchNorm2d       │    256 │
│ 28 │ net.blocks.1.norm2         │ BatchNorm2d       │    256 │
│ 29 │ net.blocks.1.drop          │ Dropout           │      0 │
│ 30 │ net.blocks.2               │ ResidualBlock     │  295 K │
│ 31 │ net.blocks.2.activation    │ LeakyReLU         │      0 │
│ 32 │ net.blocks.2.conv1         │ PeriodicConv2D    │  147 K │
│ 33 │ net.blocks.2.conv1.padding │ PeriodicPadding2D │      0 │
│ 34 │ net.blocks.2.conv1.conv    │ Conv2d            │  147 K │
│ 35 │ net.blocks.2.conv2         │ PeriodicConv2D    │  147 K │
│ 36 │ net.blocks.2.conv2.padding │ PeriodicPadding2D │      0 │
│ 37 │ net.blocks.2.conv2.conv    │ Conv2d            │  147 K │
│ 38 │ net.blocks.2.shortcut      │ Identity          │      0 │
│ 39 │ net.blocks.2.norm1         │ BatchNorm2d       │    256 │
│ 40 │ net.blocks.2.norm2         │ BatchNorm2d       │    256 │
│ 41 │ net.blocks.2.drop          │ Dropout           │      0 │
│ 42 │ net.blocks.3               │ ResidualBlock     │  295 K │
│ 43 │ net.blocks.3.activation    │ LeakyReLU         │      0 │
│ 44 │ net.blocks.3.conv1         │ PeriodicConv2D    │  147 K │
│ 45 │ net.blocks.3.conv1.padding │ PeriodicPadding2D │      0 │
│ 46 │ net.blocks.3.conv1.conv    │ Conv2d            │  147 K │
│ 47 │ net.blocks.3.conv2         │ PeriodicConv2D    │  147 K │
│ 48 │ net.blocks.3.conv2.padding │ PeriodicPadding2D │      0 │
│ 49 │ net.blocks.3.conv2.conv    │ Conv2d            │  147 K │
│ 50 │ net.blocks.3.shortcut      │ Identity          │      0 │
│ 51 │ net.blocks.3.norm1         │ BatchNorm2d       │    256 │
│ 52 │ net.blocks.3.norm2         │ BatchNorm2d       │    256 │
│ 53 │ net.blocks.3.drop          │ Dropout           │      0 │
│ 54 │ net.norm                   │ BatchNorm2d       │    256 │
│ 55 │ net.final                  │ PeriodicConv2D    │  6.3 K │
│ 56 │ net.final.padding          │ PeriodicPadding2D │      0 │
│ 57 │ net.final.conv             │ Conv2d            │  6.3 K │
│ 58 │ denormalization            │

Trainable params: 1.2 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.2 M                                                                          
Total estimated model params size (MB): 2

Output()

In [20]:
fit_lin_reg_baseline(era5_model_module, era5_data_module, reg_hparam=0.0)

# Data

Days: 5, Var: temp 850, model: resnet

train_start_year = 1979 / 1851, val_start_year = 2011, test_start_year = 2013, end_year = 2014

In [13]:
# cmip -> era
cmip_trainer.test(cmip_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.6448484416599636              │
│      test/w_rmse_temperature_850_day_5      │             3.9293335785199504              │
│ test_climatology_baseline/w_rmse_temperatu… │             5.5931557615231045              │
│ test_persistence_baseline/w_rmse_temperatu… │              3.743477762822621              │
│ test_ridge_regression_baseline/w_rmse_temp… │             3.7446136846837055              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [21]:
# era -> era
era5_trainer.test(era5_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.6584120327752235              │
│      test/w_rmse_temperature_850_day_5      │             3.6573641252154023              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.377469024461447              │
│ test_persistence_baseline/w_rmse_temperatu… │             3.6627157649810713              │
│ test_ridge_regression_baseline/w_rmse_temp… │              3.662976853861721              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [15]:
# cmip -> cmip
cmip_trainer.test(cmip_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5128192263688587              │
│      test/w_rmse_temperature_850_day_5      │              4.553549147934444              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.64543811818881               │
│ test_persistence_baseline/w_rmse_temperatu… │             4.7183777055751275              │
│ test_ridge_regression_baseline/w_rmse_temp… │              4.719226878406928              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [22]:
# era -> cmip
era5_trainer.test(era5_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5552452606953945              │
│      test/w_rmse_temperature_850_day_5      │             4.4955920242666485              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.568385305731949              │
│ test_persistence_baseline/w_rmse_temperatu… │              4.616583280907403              │
│ test_ridge_regression_baseline/w_rmse_temp… │              4.61687780971538               │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

## Unet??

In [10]:
cmip_unet_model_module = load_model(name = "unet", task = "forecasting", model_kwargs = cmip_model_kwargs, optim_kwargs = optim_kwargs)
era5_unet_model_module = load_model(name = "unet", task = "forecasting", model_kwargs = era5_model_kwargs, optim_kwargs = optim_kwargs)

In [13]:
set_climatology(era5_unet_model_module, era5_data_module)
set_climatology(cmip_unet_model_module, cmip6_data_module)

In [16]:
cmip_unet_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
cmip_unet_trainer.fit(cmip_unet_model_module, cmip6_data_module)

/home/seongbin/miniconda3/envs/climate_pretrain/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /data0/ckpts/seongbin/cross-train-z500/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                          ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                           │ Unet              │  250 M │
│ 1   │ net.activation                │ LeakyReLU         │      0 │
│ 2   │ net.image_proj                │ PeriodicConv2D    │  3.2 K │
│ 3   │ net.image_proj.padding        │ PeriodicPadding2D │      0 │
│ 4   │ net.image_proj.conv           │ Conv2d            │  3.2 K │
│ 5   │ net.down                      │ ModuleList        │ 75.4 M │
│ 6   │ net.down.0                    │ DownBlock         │ 74.1 K │
│ 7   │ net.down.0.res                │ ResidualBlock     │ 74.1 K │
│ 8   │ net.down.0.res.activation     │ LeakyReLU         │      0 │
│ 9   │ net.down.0.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 10  │ net.down.0.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 11  │ net.down.0.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 12  │ net.down.0.res.conv2          │ PeriodicConv2D    │ 36.9 K │
│ 13  │ net.down.0.res.conv2.padding  │ PeriodicPadding2D │      0 │
│ 14  │ net.down.0.res.conv2.conv     │ Conv2d            │ 36.9 K │
│ 15  │ net.down.0.res.shortcut       │ Identity          │      0 │
│ 16  │ net.down.0.res.norm1          │ BatchNorm2d       │    128 │
│ 17  │ net.down.0.res.norm2          │ BatchNorm2d       │    128 │
│ 18  │ net.down.0.res.drop           │ Dropout           │      0 │
│ 19  │ net.down.0.attn               │ Identity          │      0 │
│ 20  │ net.down.1                    │ DownBlock         │ 74.1 K │
│ 21  │ net.down.1.res                │ ResidualBlock     │ 74.1 K │
│ 22  │ net.down.1.res.activation     │ LeakyReLU         │      0 │
│ 23  │ net.down.1.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 24  │ net.down.1.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 25  │ net.down.1.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 26  │ net.down.1.res.conv2          │ PeriodicConv2D    │ 36.9 K │
│ 27  │ net.down.1.res.conv2.padding  │ PeriodicPadding2D │      0 │
│ 28  │ net.down.1.res.conv2.conv     │ Conv2d            │ 36.9 K │
│ 29  │ net.down.1.res.shortcut       │ Identity          │      0 │
│ 30  │ net.down.1.res.norm1          │ BatchNorm2d       │    128 │
│ 31  │ net.down.1.res.norm2          │ BatchNorm2d       │    128 │
│ 32  │ net.down.1.res.drop           │ Dropout           │      0 │
│ 33  │ net.down.1.attn               │ Identity          │      0 │
│ 34  │ net.down.2                    │ DownBlock         │ 74.1 K │
│ 35  │ net.down.2.res                │ ResidualBlock     │ 74.1 K │
│ 36  │ net.down.2.res.activation     │ LeakyReLU         │      0 │
│ 37  │ net.down.2.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 38  │ net.down.2.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 39  │ net.down.2.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 40  │ net.down.2.res.conv2          │ PeriodicConv2D    │ 36.9 K │
│ 41  │ net.down.2.res.conv2.padding  │ PeriodicPadding2D │      0 │
│ 42  │ net.down.2.res.conv2.conv     │ Conv2d            │ 36.9 K │
│ 43  │ net.down.2.res.shortcut       │ Identity          │      0 │
│ 44  │ net.down.2.res.norm1          │ BatchNorm2d       │    128 │
│ 45  │ net.down.2.res.norm2          │ BatchNorm2d       │    128 │
│ 46  │ net.down.2.res.drop           │ Dropout           │      0 │
│ 47  │ net.down.2.attn               │ Identity          │      0 │
│ 48  │ net.down.3                    │ DownBlock         │ 74.1 K │
│ 49  │ net.down.3.res                │ ResidualBlock     │ 74.1 K │
│ 50  │ net.down.3.res.activation     │ LeakyReLU         │      0 │
│ 51  │ net.down.3.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 52  │ net.down.3.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 53  │ net.down.3.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 54  │ net.down.3.res.conv2          │ PeriodicConv2D    │ 36.9 K 

Trainable params: 250 M                                                                      
Non-trainable params: 0                                                                      
Total params: 250 M                                                                          
Total estimated model params size (MB): 501

Output()

In [19]:
era5_unet_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
era5_unet_trainer.fit(era5_unet_model_module, era5_data_module)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                          ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                           │ Unet              │  250 M │
│ 1   │ net.activation                │ LeakyReLU         │      0 │
│ 2   │ net.image_proj                │ PeriodicConv2D    │  3.2 K │
│ 3   │ net.image_proj.padding        │ PeriodicPadding2D │      0 │
│ 4   │ net.image_proj.conv           │ Conv2d            │  3.2 K │
│ 5   │ net.down                      │ ModuleList        │ 75.4 M │
│ 6   │ net.down.0                    │ DownBlock         │ 74.1 K │
│ 7   │ net.down.0.res                │ ResidualBlock     │ 74.1 K │
│ 8   │ net.down.0.res.activation     │ LeakyReLU         │      0 │
│ 9   │ net.down.0.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 10  │ net.down.0.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 11  │ net.down.0.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 12  │ net.down.0.res.conv2          │ PeriodicConv2D    │ 36.9 K │
│ 13  │ net.down.0.res.conv2.padding  │ PeriodicPadding2D │      0 │
│ 14  │ net.down.0.res.conv2.conv     │ Conv2d            │ 36.9 K │
│ 15  │ net.down.0.res.shortcut       │ Identity          │      0 │
│ 16  │ net.down.0.res.norm1          │ BatchNorm2d       │    128 │
│ 17  │ net.down.0.res.norm2          │ BatchNorm2d       │    128 │
│ 18  │ net.down.0.res.drop           │ Dropout           │      0 │
│ 19  │ net.down.0.attn               │ Identity          │      0 │
│ 20  │ net.down.1                    │ DownBlock         │ 74.1 K │
│ 21  │ net.down.1.res                │ ResidualBlock     │ 74.1 K │
│ 22  │ net.down.1.res.activation     │ LeakyReLU         │      0 │
│ 23  │ net.down.1.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 24  │ net.down.1.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 25  │ net.down.1.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 26  │ net.down.1.res.conv2          │ PeriodicConv2D    │ 36.9 K │
│ 27  │ net.down.1.res.conv2.padding  │ PeriodicPadding2D │      0 │
│ 28  │ net.down.1.res.conv2.conv     │ Conv2d            │ 36.9 K │
│ 29  │ net.down.1.res.shortcut       │ Identity          │      0 │
│ 30  │ net.down.1.res.norm1          │ BatchNorm2d       │    128 │
│ 31  │ net.down.1.res.norm2          │ BatchNorm2d       │    128 │
│ 32  │ net.down.1.res.drop           │ Dropout           │      0 │
│ 33  │ net.down.1.attn               │ Identity          │      0 │
│ 34  │ net.down.2                    │ DownBlock         │ 74.1 K │
│ 35  │ net.down.2.res                │ ResidualBlock     │ 74.1 K │
│ 36  │ net.down.2.res.activation     │ LeakyReLU         │      0 │
│ 37  │ net.down.2.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 38  │ net.down.2.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 39  │ net.down.2.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 40  │ net.down.2.res.conv2          │ PeriodicConv2D    │ 36.9 K │
│ 41  │ net.down.2.res.conv2.padding  │ PeriodicPadding2D │      0 │
│ 42  │ net.down.2.res.conv2.conv     │ Conv2d            │ 36.9 K │
│ 43  │ net.down.2.res.shortcut       │ Identity          │      0 │
│ 44  │ net.down.2.res.norm1          │ BatchNorm2d       │    128 │
│ 45  │ net.down.2.res.norm2          │ BatchNorm2d       │    128 │
│ 46  │ net.down.2.res.drop           │ Dropout           │      0 │
│ 47  │ net.down.2.attn               │ Identity          │      0 │
│ 48  │ net.down.3                    │ DownBlock         │ 74.1 K │
│ 49  │ net.down.3.res                │ ResidualBlock     │ 74.1 K │
│ 50  │ net.down.3.res.activation     │ LeakyReLU         │      0 │
│ 51  │ net.down.3.res.conv1          │ PeriodicConv2D    │ 36.9 K │
│ 52  │ net.down.3.res.conv1.padding  │ PeriodicPadding2D │      0 │
│ 53  │ net.down.3.res.conv1.conv     │ Conv2d            │ 36.9 K │
│ 54  │ net.down.3.res.conv2          │ PeriodicConv2D    │ 36.9 K 

Trainable params: 250 M                                                                      
Non-trainable params: 0                                                                      
Total params: 250 M                                                                          
Total estimated model params size (MB): 501

Output()

In [20]:
# era -> era
era5_unet_trainer.test(era5_unet_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.6654764416472941              │
│      test/w_rmse_temperature_850_day_5      │              3.698711336818152              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.377469024461447              │
│ test_persistence_baseline/w_rmse_temperatu… │             3.6627157649810713              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [17]:
# cmip -> era
cmip_unet_trainer.test(cmip_unet_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5802755738988595              │
│      test/w_rmse_temperature_850_day_5      │              4.280544086593637              │
│ test_climatology_baseline/w_rmse_temperatu… │             5.5931557615231045              │
│ test_persistence_baseline/w_rmse_temperatu… │              3.743477762822621              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [18]:
# cmip -> cmip
cmip_unet_trainer.test(cmip_unet_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.6801928141696266              │
│      test/w_rmse_temperature_850_day_5      │             3.7886415897554206              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.64543811818881               │
│ test_persistence_baseline/w_rmse_temperatu… │             4.7183777055751275              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [21]:
# era -> cmip
era5_unet_trainer.test(era5_unet_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5675370522063884              │
│      test/w_rmse_temperature_850_day_5      │              4.501253867191024              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.568385305731949              │
│ test_persistence_baseline/w_rmse_temperatu… │              4.616583280907403              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

## ViT???

In [22]:
cmip_vit_model_kwargs = {
    "img_size": [32, 64],
    "in_vars": ["temperature_850"],
    "out_vars": ["temperature_850"],
}

era5_vit_model_kwargs = {
    "img_size": [32, 64],
    "in_vars": ["temperature_850"],
    "out_vars": ["temperature_850"],
}

# model_kwargs = {
#     "in_vars": ["2m_temperature"],
#     "out_vars": ["2m_temperature"],
#     "img_size": [32, 64]
# }

In [23]:
cmip_vit_model_module = load_model(name = "vit", task = "forecasting", model_kwargs = cmip_vit_model_kwargs, optim_kwargs = optim_kwargs)
era5_vit_model_module = load_model(name = "vit", task = "forecasting", model_kwargs = era5_vit_model_kwargs, optim_kwargs = optim_kwargs)

In [24]:
set_climatology(era5_vit_model_module, era5_data_module)
set_climatology(cmip_vit_model_module, cmip6_data_module)

In [25]:
cmip_vit_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
cmip_vit_trainer.fit(cmip_vit_model_module, cmip6_data_module)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                         ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                          │ VisionTransformer │  311 M │
│ 1   │ net.patch_embed              │ PatchEmbed        │  263 K │
│ 2   │ net.patch_embed.proj         │ Conv2d            │  263 K │
│ 3   │ net.patch_embed.norm         │ Identity          │      0 │
│ 4   │ net.pos_drop                 │ Dropout           │      0 │
│ 5   │ net.blocks                   │ ModuleList        │  302 M │
│ 6   │ net.blocks.0                 │ Block             │ 12.6 M │
│ 7   │ net.blocks.0.norm1           │ LayerNorm         │  2.0 K │
│ 8   │ net.blocks.0.attn            │ Attention         │  4.2 M │
│ 9   │ net.blocks.0.attn.qkv        │ Linear            │  3.1 M │
│ 10  │ net.blocks.0.attn.attn_drop  │ Dropout           │      0 │
│ 11  │ net.blocks.0.attn.proj       │ Linear            │  1.0 M │
│ 12  │ net.blocks.0.attn.proj_drop  │ Dropout           │      0 │
│ 13  │ net.blocks.0.drop_path       │ Identity          │      0 │
│ 14  │ net.blocks.0.norm2           │ LayerNorm         │  2.0 K │
│ 15  │ net.blocks.0.mlp             │ Mlp               │  8.4 M │
│ 16  │ net.blocks.0.mlp.fc1         │ Linear            │  4.2 M │
│ 17  │ net.blocks.0.mlp.act         │ GELU              │      0 │
│ 18  │ net.blocks.0.mlp.drop1       │ Dropout           │      0 │
│ 19  │ net.blocks.0.mlp.fc2         │ Linear            │  4.2 M │
│ 20  │ net.blocks.0.mlp.drop2       │ Dropout           │      0 │
│ 21  │ net.blocks.1                 │ Block             │ 12.6 M │
│ 22  │ net.blocks.1.norm1           │ LayerNorm         │  2.0 K │
│ 23  │ net.blocks.1.attn            │ Attention         │  4.2 M │
│ 24  │ net.blocks.1.attn.qkv        │ Linear            │  3.1 M │
│ 25  │ net.blocks.1.attn.attn_drop  │ Dropout           │      0 │
│ 26  │ net.blocks.1.attn.proj       │ Linear            │  1.0 M │
│ 27  │ net.blocks.1.attn.proj_drop  │ Dropout           │      0 │
│ 28  │ net.blocks.1.drop_path       │ DropPath          │      0 │
│ 29  │ net.blocks.1.norm2           │ LayerNorm         │  2.0 K │
│ 30  │ net.blocks.1.mlp             │ Mlp               │  8.4 M │
│ 31  │ net.blocks.1.mlp.fc1         │ Linear            │  4.2 M │
│ 32  │ net.blocks.1.mlp.act         │ GELU              │      0 │
│ 33  │ net.blocks.1.mlp.drop1       │ Dropout           │      0 │
│ 34  │ net.blocks.1.mlp.fc2         │ Linear            │  4.2 M │
│ 35  │ net.blocks.1.mlp.drop2       │ Dropout           │      0 │
│ 36  │ net.blocks.2                 │ Block             │ 12.6 M │
│ 37  │ net.blocks.2.norm1           │ LayerNorm         │  2.0 K │
│ 38  │ net.blocks.2.attn            │ Attention         │  4.2 M │
│ 39  │ net.blocks.2.attn.qkv        │ Linear            │  3.1 M │
│ 40  │ net.blocks.2.attn.attn_drop  │ Dropout           │      0 │
│ 41  │ net.blocks.2.attn.proj       │ Linear            │  1.0 M │
│ 42  │ net.blocks.2.attn.proj_drop  │ Dropout           │      0 │
│ 43  │ net.blocks.2.drop_path       │ DropPath          │      0 │
│ 44  │ net.blocks.2.norm2           │ LayerNorm         │  2.0 K │
│ 45  │ net.blocks.2.mlp             │ Mlp               │  8.4 M │
│ 46  │ net.blocks.2.mlp.fc1         │ Linear            │  4.2 M │
│ 47  │ net.blocks.2.mlp.act         │ GELU              │      0 │
│ 48  │ net.blocks.2.mlp.drop1       │ Dropout           │      0 │
│ 49  │ net.blocks.2.mlp.fc2         │ Linear            │  4.2 M │
│ 50  │ net.blocks.2.mlp.drop2       │ Dropout           │      0 │
│ 51  │ net.blocks.3                 │ Block             │ 12.6 M │
│ 52  │ net.blocks.3.norm1           │ LayerNorm         │  2.0 K │
│ 53  │ net.blocks.3.attn            │ Attention         │  4.2 M │
│ 54  │ net.blocks.3.attn.qkv        │ Linear            │  3.1 M │
│ 55  │ net.blocks.3.attn.attn_drop  │ Dropout          

Trainable params: 311 M                                                                      
Non-trainable params: 8.2 K                                                                  
Total params: 311 M                                                                          
Total estimated model params size (MB): 622

Output()

In [26]:
era5_vit_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
era5_vit_trainer.fit(era5_vit_model_module, era5_data_module)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                         ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                          │ VisionTransformer │  311 M │
│ 1   │ net.patch_embed              │ PatchEmbed        │  263 K │
│ 2   │ net.patch_embed.proj         │ Conv2d            │  263 K │
│ 3   │ net.patch_embed.norm         │ Identity          │      0 │
│ 4   │ net.pos_drop                 │ Dropout           │      0 │
│ 5   │ net.blocks                   │ ModuleList        │  302 M │
│ 6   │ net.blocks.0                 │ Block             │ 12.6 M │
│ 7   │ net.blocks.0.norm1           │ LayerNorm         │  2.0 K │
│ 8   │ net.blocks.0.attn            │ Attention         │  4.2 M │
│ 9   │ net.blocks.0.attn.qkv        │ Linear            │  3.1 M │
│ 10  │ net.blocks.0.attn.attn_drop  │ Dropout           │      0 │
│ 11  │ net.blocks.0.attn.proj       │ Linear            │  1.0 M │
│ 12  │ net.blocks.0.attn.proj_drop  │ Dropout           │      0 │
│ 13  │ net.blocks.0.drop_path       │ Identity          │      0 │
│ 14  │ net.blocks.0.norm2           │ LayerNorm         │  2.0 K │
│ 15  │ net.blocks.0.mlp             │ Mlp               │  8.4 M │
│ 16  │ net.blocks.0.mlp.fc1         │ Linear            │  4.2 M │
│ 17  │ net.blocks.0.mlp.act         │ GELU              │      0 │
│ 18  │ net.blocks.0.mlp.drop1       │ Dropout           │      0 │
│ 19  │ net.blocks.0.mlp.fc2         │ Linear            │  4.2 M │
│ 20  │ net.blocks.0.mlp.drop2       │ Dropout           │      0 │
│ 21  │ net.blocks.1                 │ Block             │ 12.6 M │
│ 22  │ net.blocks.1.norm1           │ LayerNorm         │  2.0 K │
│ 23  │ net.blocks.1.attn            │ Attention         │  4.2 M │
│ 24  │ net.blocks.1.attn.qkv        │ Linear            │  3.1 M │
│ 25  │ net.blocks.1.attn.attn_drop  │ Dropout           │      0 │
│ 26  │ net.blocks.1.attn.proj       │ Linear            │  1.0 M │
│ 27  │ net.blocks.1.attn.proj_drop  │ Dropout           │      0 │
│ 28  │ net.blocks.1.drop_path       │ DropPath          │      0 │
│ 29  │ net.blocks.1.norm2           │ LayerNorm         │  2.0 K │
│ 30  │ net.blocks.1.mlp             │ Mlp               │  8.4 M │
│ 31  │ net.blocks.1.mlp.fc1         │ Linear            │  4.2 M │
│ 32  │ net.blocks.1.mlp.act         │ GELU              │      0 │
│ 33  │ net.blocks.1.mlp.drop1       │ Dropout           │      0 │
│ 34  │ net.blocks.1.mlp.fc2         │ Linear            │  4.2 M │
│ 35  │ net.blocks.1.mlp.drop2       │ Dropout           │      0 │
│ 36  │ net.blocks.2                 │ Block             │ 12.6 M │
│ 37  │ net.blocks.2.norm1           │ LayerNorm         │  2.0 K │
│ 38  │ net.blocks.2.attn            │ Attention         │  4.2 M │
│ 39  │ net.blocks.2.attn.qkv        │ Linear            │  3.1 M │
│ 40  │ net.blocks.2.attn.attn_drop  │ Dropout           │      0 │
│ 41  │ net.blocks.2.attn.proj       │ Linear            │  1.0 M │
│ 42  │ net.blocks.2.attn.proj_drop  │ Dropout           │      0 │
│ 43  │ net.blocks.2.drop_path       │ DropPath          │      0 │
│ 44  │ net.blocks.2.norm2           │ LayerNorm         │  2.0 K │
│ 45  │ net.blocks.2.mlp             │ Mlp               │  8.4 M │
│ 46  │ net.blocks.2.mlp.fc1         │ Linear            │  4.2 M │
│ 47  │ net.blocks.2.mlp.act         │ GELU              │      0 │
│ 48  │ net.blocks.2.mlp.drop1       │ Dropout           │      0 │
│ 49  │ net.blocks.2.mlp.fc2         │ Linear            │  4.2 M │
│ 50  │ net.blocks.2.mlp.drop2       │ Dropout           │      0 │
│ 51  │ net.blocks.3                 │ Block             │ 12.6 M │
│ 52  │ net.blocks.3.norm1           │ LayerNorm         │  2.0 K │
│ 53  │ net.blocks.3.attn            │ Attention         │  4.2 M │
│ 54  │ net.blocks.3.attn.qkv        │ Linear            │  3.1 M │
│ 55  │ net.blocks.3.attn.attn_drop  │ Dropout          

Trainable params: 311 M                                                                      
Non-trainable params: 8.2 K                                                                  
Total params: 311 M                                                                          
Total estimated model params size (MB): 622

Output()

In [27]:
# cmip -> era
cmip_vit_trainer.test(cmip_vit_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.4762501114412623              │
│      test/w_rmse_temperature_850_day_5      │              4.762497936542316              │
│ test_climatology_baseline/w_rmse_temperatu… │             5.5931557615231045              │
│ test_persistence_baseline/w_rmse_temperatu… │              3.743477762822621              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [28]:
# cmip -> cmip
cmip_vit_trainer.test(cmip_vit_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5595388788761634              │
│      test/w_rmse_temperature_850_day_5      │              4.34326734004565               │
│ test_climatology_baseline/w_rmse_temperatu… │              5.64543811818881               │
│ test_persistence_baseline/w_rmse_temperatu… │             4.7183777055751275              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [29]:
# era -> era
era5_vit_trainer.test(era5_vit_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5707147691453198              │
│      test/w_rmse_temperature_850_day_5      │              4.026696913489575              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.377469024461447              │
│ test_persistence_baseline/w_rmse_temperatu… │             3.6627157649810713              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [30]:
# era -> cmip
era5_vit_trainer.test(era5_vit_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_temperature_850_day_5        │             0.5287943395398247              │
│      test/w_rmse_temperature_850_day_5      │              4.519943759424461              │
│ test_climatology_baseline/w_rmse_temperatu… │              5.568385305731949              │
│ test_persistence_baseline/w_rmse_temperatu… │              4.616583280907403              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘